In [1]:
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from pykrige.rk import RegressionKriging
import geopandas as gpd
import rasterio
import rioxarray
import xarray

In [2]:
# load the bathy points

pts_gdf = gpd.read_file('../data/test_sites/florida_keys/all_bathy_pts.gpkg',mode='r')
# pts_gdf = pts_gdf.drop(columns=['fema_elev']).dropna()

In [3]:
# keep points within a half meter of the estimated seafloor
pts_gdf = pts_gdf[abs(pts_gdf.Z_g - pts_gdf.z_kde) < 0.005]

In [8]:
pts_gdf

,index,X,Y,Z,Z_g,geoid_corr,tide_ocean_corr,geof2m_corr,delta_time,oc_sig_conf,...,p_vec_elev,dist_or,easting,northing,gebco_elev,sea_level_interp,Z_refr,z_kde,kde_val,geometry
168,2918,-81.065271,24.653398,-35.730679,-11.475018,-24.187841,-0.129566,0.061747,2021-08-31T18:40:05.508000,1,...,1.563195,118.566088,493395.167609,2.726571e+06,-6.0,0.122495,-8.527498,-11.475018,0.052583,POINT (493395.168 2726571.482)
175,2965,-81.065274,24.653424,-35.699814,-11.444153,-24.187841,-0.129566,0.061747,2021-08-31T18:40:05.509000,1,...,1.563195,121.405757,493394.908090,2.726574e+06,-6.0,0.127371,-8.503238,-11.444153,0.053044,POINT (493394.908 2726574.310)
197,890,-81.066127,24.653047,-37.299885,-13.044371,-24.187752,-0.129510,0.061748,2021-08-31T18:40:05.165000,1,...,1.563281,125.777737,493308.507477,2.726533e+06,-6.0,0.127930,-9.696614,-13.044371,0.032844,POINT (493308.507 2726532.601)
374,3655,-81.065312,24.653800,-35.602631,-11.347387,-24.187220,-0.129769,0.061745,2021-08-31T18:40:05.515000,1,...,1.563195,163.303501,493391.018576,2.726616e+06,-6.0,0.106098,-8.436472,-11.347387,0.057179,POINT (493391.019 2726616.027)
603,1517,-81.066227,24.654012,-35.772236,-11.517780,-24.186176,-0.130023,0.061744,2021-08-31T18:40:05.180000,1,...,1.563282,215.084795,493298.419231,2.726640e+06,-6.0,0.074926,-8.571480,-11.517780,0.032908,POINT (493298.419 2726639.540)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973988,51094,-80.876169,24.831108,-28.329996,-3.869675,-24.580956,0.059795,0.060839,2021-12-29T12:56:07.849000,3,...,1.550709,27514.756904,512512.768043,2.746252e+06,-2.0,-0.044630,-2.897755,-3.869675,0.099662,POINT (512512.768 2746252.491)
974091,799,-80.875160,24.830627,-28.571096,-4.171381,-24.582289,0.121734,0.060841,2018-10-17T08:53:07.988000,4,...,1.564340,27547.501057,512614.753235,2.746199e+06,-2.0,-0.105101,-3.137921,-4.171381,0.209224,POINT (512614.753 2746199.271)
974134,732,-80.875139,24.830832,-28.545771,-4.145792,-24.582535,0.121715,0.060840,2018-10-17T08:53:07.985000,4,...,1.564340,27565.298818,512616.878136,2.746222e+06,-2.0,-0.114143,-3.121134,-4.142960,0.216877,POINT (512616.878 2746222.028)
974150,51463,-80.876250,24.831890,-28.143761,-3.682980,-24.581715,0.060099,0.060836,2021-12-29T12:56:07.861000,2,...,1.550709,27571.365029,512504.515907,2.746339e+06,-2.0,-0.050741,-2.760050,-3.682980,0.114116,POINT (512504.516 2746339.013)


In [10]:
pts_gdf = pts_gdf.sort_values('kde_val',ascending=False).iloc[:3000]

In [11]:
ras = rasterio.open('../data/resample_test/bilinear.tif')

In [12]:
ar = rioxarray.open_rasterio(ras)
gridx = ar.x.data
gridy = ar.y.data

In [14]:
# going to randomly downsample the lidar point dataframe to make the UK model faster

# pts_gdf = pts_gdf.sample(1000)
pts_gdf.to_file('../data/resample_test/keys_testpts.gpkg')

x_loc = pts_gdf.geometry.x.to_numpy()
y_loc = pts_gdf.geometry.y.to_numpy()
z_elev = pts_gdf.Z_g.to_numpy()

/home/mlinds/mambaforge/envs/thesis_env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [15]:
# x_loc = x_loc - x_loc.mean()
# y_loc = y_loc - y_loc.mean()

In [16]:
# okmodel = OrdinaryKriging(
#     x=x_loc,
#     y=y_loc,
#     z=z_elev,
#     variogram_model='linear',
#     verbose=True,
#     coordinates_type = "euclidean",
# )

ukmodel = UniversalKriging(
    x=x_loc,
    y=y_loc,
    z=z_elev,
    variogram_model='linear',
    verbose=True,
    # coordinates_type = "euclidean",
)
    

Adjusting data for anisotropy...
Initializing variogram model...
Using 'linear' Variogram Model
Slope: 0.00017073684906286272
Nugget: 5.094711972632987 

Calculating statistics on variogram model fit...
Q1 = 0.060736256953489456
Q2 = 0.23869150996400137
cR = 1.2836123004375 

Initializing drift terms...


In [17]:
z,ss = ukmodel.execute('grid',gridx,gridy)

Executing Universal Kriging...



In [10]:
ras.dtypes

('float32',)

In [11]:
# xarray and rioxarray are too fuckin complicated, going to just use rasterio if I can

with rasterio.open('../data/resample_test/interp.tif',mode='w+',crs=ras.crs,width=ras.width,height=ras.height,count=2,dtype=ras.dtypes[0],transform=ras.transform,) as rasout:
    rasout.write(z,1)
    rasout.write(ss,2)


In [12]:
masks= ras.read_masks(1)